In [7]:
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from transformers import pipeline

In [22]:
# Load your KB from a text file
with open("Film_Summary.txt", "r", encoding="utf-8") as f:
    kb_text = f.read()

# Simple fixed-size chunking
chunk_size = 300
chunks = [kb_text[i:i+chunk_size] for i in range(0, len(kb_text), chunk_size)]
print(f"Total chunks: {len(chunks)}")

Total chunks: 5


In [23]:
embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = embedder.encode(chunks, convert_to_numpy=True, show_progress_bar=True)

# Normalize embeddings for cosine similarity search
embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)


Batches: 100%|██████████| 1/1 [00:00<00:00, 20.10it/s]


In [24]:
# Determine embedding dimension
dimension = embeddings.shape[1]

# Create an index that uses Inner Product (cosine similarity for normalized vectors)
index = faiss.IndexFlatIP(dimension)

# Add embeddings to index
index.add(embeddings)
print("FAISS index populated with", index.ntotal, "vectors.")

FAISS index populated with 5 vectors.


In [25]:
def retrieve_relevant_chunks(query, top_k=2):
    # Encode and normalize query
    query_embedding = embedder.encode([query], convert_to_numpy=True)
    query_embedding = query_embedding / np.linalg.norm(query_embedding, axis=1, keepdims=True)

    # Search for the top_k most similar chunks
    scores, indices = index.search(query_embedding, top_k)

    # Retrieve matching text chunks
    retrieved_chunks = [chunks[i] for i in indices[0]]
    return retrieved_chunks, scores[0]

In [26]:
model_name = "google/flan-t5-small"
generator = pipeline("text2text-generation", model=model_name)

Device set to use cpu


In [52]:
def generate_answer(query):
    context_chunks, scores = retrieve_relevant_chunks(query, top_k=2)
    context = "\n".join(context_chunks)
    
    prompt = f"Use the context below to help answer the question. If the answer cannot be inferred by the context or general knowledge, respond with 'I don't know'.\n\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"
    
    response = generator(prompt, max_length=500, do_sample=False)
    return response[0]["generated_text"]

In [55]:
query = "What is the name of the movie?"
answer = generate_answer(query)

print("Query:", query)
print("\nAnswer:", answer)

Query: What is the name of the movie?

Answer: Sympathy for Lady Vengeance
